In [1]:
%load_ext autoreload 
%autoreload 1

In [3]:
import functools
import gym
from Config import Config
# from util import train
from Models import ActorCritic
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Memory import Memory
from baselines.common.cmd_util import make_env
from baselines.common.atari_wrappers import wrap_deepmind, make_atari
import matplotlib.pyplot as plt

In [7]:
env = gym.make('CarRacing-v0')
env.observation_space.shape
env.action_space

Box(3,)

In [ ]:
config = Config(gym.make('CarRacing-v0'))
config = Config(env)

config.update_every = 500
config.num_learn = 4
config.win_condition = 230
config.n_episodes = 1000
config.max_t = 700

config.Memory = Memory
config.Model = ActorCritic
config.head_model = functools.partial(cnn_head_model, config)
config.actor_model = functools.partial(actor_model, config)
config.critic_model = functools.partial(critic_model, config)

In [ ]:
import torch
import copy

env = copy.deepcopy(config.env)
state = env.reset()
print(torch.FloatTensor(state).shape)
# agent = PPO(config)

# action, log_prob = agent.act(torch.FloatTensor(state))
# print("[unsqueezed] Action space shape: {}".format(action.shape))
# print("[unsqueezed] Log Probabilities: {}".format(log_prob))

# action, log_prob = agent.act(torch.FloatTensor(state))
# print("[unsqueezed] Action space shape: {}".format(action.shape))
# print("[unsqueezed] Log Probabilities: {}".format(log_prob))

In [ ]:
import copy
import gym
import torch
import numpy as np
from collections import deque
from PPO import PPO
from Config import Config
import pdb

def get_state(obs):
    state = np.array(obs)
    state = state.transpose((2, 0, 1))
    state = torch.FloatTensor(state)
    return state.unsqueeze(0)

def get_save_state(obs):
    state = np.array(obs)
    state = state.transpose((2, 0, 1))
    state = torch.FloatTensor(state)
    return state

def train(config):
    env = copy.deepcopy(config.env)
    steps = 0
    scores_deque = deque(maxlen=100)
    scores = []
    average_scores = []
    max_score = -np.Inf

    agent = PPO(config)

    for i_episode in range(1, config.n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(config.max_t):
            steps += 1

            action, log_prob = agent.act(get_state(state))
            next_state, reward, done, _ = env.step(action)

            agent.mem.add(get_save_state(state), action, reward, log_prob, done)

            # Update 
            state = next_state
            score += reward


            if steps >= config.update_every:
                agent.learn(config.num_learn)
                agent.mem.clear()
                steps = 0

            if done:
                break 

        # Book Keeping
        scores_deque.append(score)
        scores.append(score)
        average_scores.append(np.mean(scores_deque))

        if i_episode % 10 == 0:
            print("\rEpisode {}	Average Score: {:.2f}	Score: {:.2f}".format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 100 == 0:
            print("\rEpisode {}	Average Score: {:.2f}".format(i_episode, np.mean(scores_deque)))   

        if np.mean(scores_deque) > config.win_condition:
            print("\nEnvironment Solved!")
            break

    return scores, average_scores


In [ ]:
scores, average_score = train(config)
plt.plot(scores)
plt.plot(average_score)
plt.show()

In [ ]:
# def get_state(obs):
#     state = np.array(obs)
#     state = state.transpose((2, 0, 1))
#     state = torch.FloatTensor(state)
#     return state.unsqueeze(0)

# def get_state_save(obs):
#     state = np.array(obs)
#     state = state.transpose((2, 0, 1))
#     state = torch.FloatTensor(state)
#     return state

# env = copy.deepcopy(config.env)
# steps = 0
# scores_deque = deque(maxlen=100)
# scores = []
# average_scores = []
# max_score = -np.Inf

# agent = PPO(config)

# for i_episode in range(1, config.n_episodes+1):
#     state = env.reset()
#     score = 0
#     for t in range(config.max_t):
#         steps += 1

#         action, log_prob = agent.act(get_state(state))
# #             print("Action space shape: {}".format(action.shape))
# #             print("Action: {}".format(action))
# #             print("Log Probabilities: {}".format(log_prob))
# #             print("Action item: {}".format(action.item()))
#         next_state, reward, done, _ = env.step(action.item())

#         agent.mem.add(get_state_save(state), action, reward, log_prob, done)

#         # Update 
#         state = next_state
#         score += reward


#         if steps >= config.update_every:
#             break

In [ ]:
# prev_states = torch.stack(agent.mem.states).to(agent.device).detach()
# prev_states.shape

In [ ]:
# action, log_prob = agent.act(prev_states)
# action.shape

In [ ]:
# state = env.reset()
# state = get_state(state)
# print(state.shape)
# action, log_prob = agent.act(state)
# action.shape
